In this notebook we give a proof of concept of unitary compiling using TFQ. 

In [48]:
%load_ext autoreload
%autoreload 2

import sympy 
import numpy as np 
import pandas as pd 
import tensorflow as tf
from utilities.circuit_database import CirqTranslater
from utilities.templates import *
from utilities.variational import Minimizer
from utilities.misc import get_qubits_involved, reindex_symbol, shift_symbols_down
import matplotlib.pyplot as plt 
import tensorflow_quantum as tfq
import cirq
from utilities.compiling import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
translator = CirqTranslater(n_qubits = 2) #system + ancilla

In [51]:
u1ss = u1_db(translator, 0, params=True)
u_to_compile_db = concatenate_dbs([u1ss]*2) ###target gates to compile
u_to_compile_db["trainable"] = False

compile_circuit, compile_circuit_db = construct_compiling_circuit(translator, u_to_compile_db)
minimizer = Minimizer(translator, mode="compiling", hamiltonian="Z")


cost, resolver, history = minimizer.minimize([compile_circuit], symbols=translator.get_symbols(compile_circuit_db))


u1s = u1_db(translator, 0, params=True)
u1s["param_value"] = -np.array(list(resolver.values()))
resu_comp, resu_db = translator.give_circuit(u1s,unresolved=False)

cc, cdb = translator.give_circuit(u_to_compile_db, unresolved=False)
c = cc.unitary()
r = resu_comp.unitary()

## phase_shift if necessary
if np.abs(np.mean(c/r) -1) > 1:
    u1s.loc[0] = u1s.loc[0].replace(to_replace=u1s["param_value"][0], value=u1s["param_value"][0] + np.pi)

In [21]:
conj = conjugate_db(translator, v_to_compile_db)

In [25]:
cc, ccdb = translator.give_circuit(conj, unresolved=False)
co, codb = translator.give_circuit(v_to_compile_db, unresolved=False)

2

In [26]:
co.unitary()

array([[-0.53686448-0.84345089j, -0.01354644-0.01355059j],
       [ 0.01354644-0.01355059j, -0.53686448+0.84345089j]])

In [24]:
cc.unitary()

array([[-0.53686448-0.84345089j, -0.01354644-0.01355059j],
       [ 0.01354644-0.01355059j, -0.53686448+0.84345089j]])

In [6]:
compiling_circuit, compiling_db = translator.give_circuit(v_to_compile_db, unresolved=False)
compiling_circuit.unitary()
resu_comp.unitary()

array([[-0.53700181+0.84357491j, -0.00305361-0.0010441j ],
       [ 0.00305361-0.0010441j , -0.53700181-0.84357491j]])

In [7]:
compiling_circuit.unitary()

array([[-0.53686448-0.84345089j, -0.01354644-0.01355059j],
       [ 0.01354644-0.01355059j, -0.53686448+0.84345089j]])

In [ ]:
for ind, gate_id in conjugate_v_to_compile.iterrows():
    if translator.number_of_cnots <= gate_id["ind"] <= translator.number_of_cnots + 3*translator.n_qubits:
        mcof = [-1,-1,1][(gate_id["ind"]-translator.number_of_cnots)//translator.n_qubits] ###this conjugates paulis  rz, rx, ry

In [2]:
def get_positional_dbs(circuit, circuit_db):

    qubits_involved = get_qubits_involved(circuit, circuit_db)
    
    gates_on_qubit = {q:[] for q in qubits_involved}
    on_qubit_order = {q:[] for q in qubits_involved}

    for order_gate, ind_gate in enumerate( circuit_db["ind"]):
        if ind_gate < translator.number_of_cnots:
            control, target = translator.indexed_cnots[str(ind_gate)]
            gates_on_qubit[control].append(ind_gate)
            gates_on_qubit[target].append(ind_gate)
            on_qubit_order[control].append(order_gate)                
            on_qubit_order[target].append(order_gate)  
        else:
            gates_on_qubit[(ind_gate-translator.n_qubits)%translator.n_qubits].append(ind_gate)
            on_qubit_order[(ind_gate-translator.n_qubits)%translator.n_qubits].append(order_gate)        
    return gates_on_qubit, on_qubit_order

In [62]:
def rule_1(translator, simplified_db, on_qubit_order, gates_on_qubit):
    simplification = False
    
    for q, qubit_gates_path in gates_on_qubit.items():
        if simplification is True:
            break
        for order_gate_on_qubit, ind_gate in enumerate(qubit_gates_path):
            if ind_gate < translator.number_of_cnots:
                control, target = translator.indexed_cnots[str(ind_gate)]
                if (q == control) and (order_gate_on_qubit == 0):
                    pos_gate_to_drop = on_qubit_order[q][order_gate_on_qubit]
                    
                    block_id = circuit_db.iloc[pos_gate_to_drop]["block_id"]
                    simplified_db.loc[int(pos_gate_to_drop)+0.1] = gate_template(translator.number_of_cnots + translator.n_qubits + control, param_value=0.0, block_id=circuit_db.iloc[0]["block_id"])
                    simplified_db.loc[int(pos_gate_to_drop)+0.11] = gate_template(translator.number_of_cnots + translator.n_qubits + target, param_value=0.0, block_id=circuit_db.iloc[0]["block_id"])
                    
                    simplified_db = simplified_db.drop(labels=[pos_gate_to_drop],axis=0)
                    
                    simplification = True
                    break
    simplified_db = simplified_db.sort_index().reset_index(drop=True)
    return simplification, simplified_db


def rule_2(translator, simplified_db, on_qubit_order, gates_on_qubit):
    simplification = False
    
    for q, qubit_gates_path in gates_on_qubit.items():
        if simplification is True:
            break
        for order_gate_on_qubit, ind_gate in enumerate(qubit_gates_path[:-1]):
            
            next_ind_gate = qubit_gates_path[order_gate_on_qubit+1]
            if (ind_gate < translator.number_of_cnots) and (ind_gate == next_ind_gate):
                control, target = translator.indexed_cnots[str(ind_gate)]
                not_gates_in_between = False
                this_qubit = q
                other_qubits = [control, target]
                other_qubits.remove(q)
                other_qubit = other_qubits[0]
                
                ## now we need to check what happens in the other_qubit
                for qord_other, ind_gate_other in enumerate(gates_on_qubit[other_qubit][:-1]):
                    if (ind_gate_other == ind_gate) and (gates_on_qubit[other_qubit][qord_other +1] == ind_gate):
                        ## if we append the CNOT for q and other_q on the same call, and also for the consecutive
                        ## note that in between there can be other calls for other qubits
                        order_call_q = on_qubit_order[q][order_gate_on_qubit]
                        order_call_other_q = on_qubit_order[other_qubit][qord_other]
                        
                        order_call_qP1 = on_qubit_order[q][order_gate_on_qubit+1]
                        order_call_other_qP1 = on_qubit_order[other_qubit][qord_other+1]
                        
                        ## then it's kosher to say they are consecutively applied (if only looking at the two qubits)
                        if (order_call_q == order_call_other_q) and (order_call_qP1 == order_call_other_qP1):
                            
                            pos_gate_to_drop = on_qubit_order[q][order_gate_on_qubit]
                            simplified_db = simplified_db.drop(labels=[pos_gate_to_drop],axis=0)
                            pos_gate_to_drop = on_qubit_order[q][order_gate_on_qubit+1]
                            simplified_db = simplified_db.drop(labels=[pos_gate_to_drop],axis=0)

                            simplification = True
                            break
                if simplification is True:
                    break
    simplified_db = simplified_db.reset_index(drop=True)
    return simplification, simplified_db



def rule_3(translator, simplified_db, on_qubit_order, gates_on_qubit):
    simplification = False
    for q, qubit_gates_path in gates_on_qubit.items():
        if simplification is True:
            break
        for order_gate_on_qubit, ind_gate in enumerate(qubit_gates_path[:-1]):
            if order_gate_on_qubit == 0 and (translator.number_of_cnots <= ind_gate< translator.number_of_cnots+ translator.n_qubits ):
                pos_gate_to_drop = on_qubit_order[q][order_gate_on_qubit]
                simplified_db = simplified_db.drop(labels=[pos_gate_to_drop],axis=0)
                simplified_db = simplified_db.reset_index(drop=True)
                simplified_db = shift_symbols_down(translator, pos_gate_to_drop, simplified_db)
                simplification = True
                break
    return simplification, simplified_db



def rule_4(translator, simplified_db, on_qubit_order, gates_on_qubit):
    """
    Repeated rotations: add the values
    """
    simplification = False
    for q, qubit_gates_path in gates_on_qubit.items():
        if simplification is True:
            break
        for order_gate_on_qubit, ind_gate in enumerate(qubit_gates_path[:-1]):
            if ind_gate>=translator.number_of_cnots:
                next_ind_gate = qubit_gates_path[order_gate_on_qubit+1]
                if next_ind_gate == ind_gate:
                    pos_gate_to_drop = on_qubit_order[q][order_gate_on_qubit]
                    pos_gate_to_add = on_qubit_order[q][order_gate_on_qubit+1]
                    
                    value_1 = simplified_db.loc[pos_gate_to_drop]["param_value"]
                    value_2 = simplified_db.loc[pos_gate_to_add]["param_value"]
                    
                    simplified_db.loc[pos_gate_to_add] = simplified_db.loc[pos_gate_to_add].replace(to_replace=value_2, value=value_1 + value_2)
                    simplified_db = simplified_db.drop(labels=[pos_gate_to_drop],axis=0)
                    simplified_db = simplified_db.reset_index(drop=True)

                    simplified_db = shift_symbols_down(translator, pos_gate_to_drop, simplified_db)
                    simplification = True
                    break
    return simplification, simplified_db

def apply_rule(original_circuit_db, rule, **kwargs):
    max_cnt = kwargs.get('max_cnt',10)
    simplified, cnt = True, 0
    original_circuit, original_circuit_db = translator.give_circuit(original_circuit_db)
    gates_on_qubit, on_qubit_order = get_positional_dbs(original_circuit, original_circuit_db)
    simplified_db = original_circuit_db.copy()
    while simplified and cnt < max_cnt:
        simplified, simplified_circuit_db = rule(translator, simplified_db, on_qubit_order, gates_on_qubit)
        circuit, simplified_db = translator.give_circuit(simplified_circuit_db)
        gates_on_qubit, on_qubit_order = get_positional_dbs(circuit, simplified_db)
        cnt+=1
    return cnt, simplified_db

In [63]:
translator = CirqTranslater(3)
db1 = u1_layer(translator)
db2 = cnot_layer(translator, block_id = 2)
circuit_db = concatenate_dbs([db1, db1])
circuit, circuit_db  = translator.give_circuit(circuit_db)
gates_on_qubit, on_qubit_order = get_positional_dbs(circuit, circuit_db)

In [64]:
cnt, simplified_db = apply_rule(circuit_db, rule_4)

In [65]:
translator.give_circuit(simplified_db)[0]

(0, 0): ───Rz(-0.655π)───Rx(-0.49π)────Rz(1.21π)────Rx(-0.49π)────Rz(1.86π)────

(0, 1): ───Rz(0.175π)────Rx(-0.817π)───Rz(-1.45π)───Rx(-0.817π)───Rz(-1.62π)───

(0, 2): ───Rz(1.82π)─────Rx(-1.82π)────Rz(1.25π)────Rx(-1.82π)────Rz(-0.57π)───